<a href="https://colab.research.google.com/drive/1FLlrv4fzFqGv4mpMGHt22x37ljxj2xTR?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">Научно-исследовательская работа</h1>

<h2 align="center"><b>"Численное моделирование механических систем"</b></h2>

<i>Выполнили:
* Липская Мария
* Карасева Ольга
* Жижченко Александр
* Пругло Иван  

Научный руководитель:
* Федоров Владимир Сергеевич

*Цели работы*:
* научиться численно моделировать механические системы
* изучить решения ограниченной задачи трёх тел
* разобраться с положением равновесия в точках Лагранжа

<h1 align="center"><b>Введение</b></h1>

_________________
  
<h2 align="center"><i><b>Основные используемые понятия</b></i></h2>

1. *Точки Лагранжа* - точки в системе из двух массивных тел, в которых третье тело с пренебрежимо малой массой, не испытывающее воздействия никаких других сил, кроме гравитационных со стороны двух первых тел, может оставаться неподвижным относительно этих тел.

1. *Полость Роша* - область вокруг объекта в двойной системе, границей которой служит эквипотенциальная поверхность, содержащая первую точку Лагранжа $L_1$

1. *Сфера Хилла* - пространство вокруг объекта, на котором он может удерживать спутники
1. *Гамильтониан* - основная функция, используемая в механике и динамике Гамильтона  
*Вид гамильтониана, используемый в работе:*

$$
  H(x, y, p_x, p_y) = \frac{(p_x + y)^2 + (p_y - x)^2}{2} - U(x, y)
$$


5. *Лагранжиан* - функция описание динамической системы через обобщённые координаты (координаты $(x, y)$ и скорости $(\dot x, \dot y)$). Общий вид:
$L(x, y, \dot x, \dot y) = T - U(x, y)$.

*(Остальные необходимые определения указаны в нужных ячейках)*

Для начала необходимо скачать компилятор Julia



In [1]:
# @title Запуск Julia
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


Подключение необходимых библиотек

In [40]:
using REPL.REPLCompletions: latex_symbols, emoji_symbols
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("Images")
using Images
using Plots

  ✓ ImageFiltering
  ✓ ImageQualityIndexes
  ✓ ImageSegmentation
  ✓ ImageCorners
  ✓ Images
  114 dependencies successfully precompiled in 296 seconds. 140 already precompiled.
  4 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


In [19]:
mu = 0.2
function u(x, y)
    r1 = ((x + mu)^2 + y^2)^0.5
    r2 = ((x - 1 + mu)^2 + y^2)
    return (x^2 + y^2) / 2 + (1 - mu) / r1 + mu / r2 + ((1 - mu) * mu) / 2
end

function h(x, y, px, py)
    """Гамильтониан системы"""
    return ((px + y)^2 + (py - x)^2) / 2 - u(x, y)
end


function diff_u_x(x, y)
    """Производная в точке от U(x, y) по x"""
    delta = 0.00001
    return (-1) * (u(x + delta / 2, y) - u(x - delta / 2, y)) / delta
end

function diff_u_y(x, y)
    """Производная в точке от U(x, y) по y"""
    delta = 0.00001
    return (-1) * (u(x, y + delta / 2) - u(x, y - delta / 2)) / delta
end

function x_dot(x, y, px, py)
    """Скорость по x"""
    return px + y
end

function y_dot(x, y, px, py)
    """Скорость по y"""
    return py - x
end

function px_dot(x, y, px, py)
    """Производная по px"""
    return py - x + diff_u_x(x, y)
end

function py_dot(x, y, px, py)
    """Производная по py"""
    return -px - y + diff_u_y(x, y)
end

function x_ddot(x, y, px, py)
    """Вторая производная по x """
    return px_dot(x, y, px, py) + y_dot(x, y, px, py)
end

function y_ddot(x, y, px, py)
    """Вторая производная по Y """
    return py_dot(x, y, px, py) - x_dot(x, y, px, py)
end

function all_dots(x, y, px, py, with_second=true)
    """Все производные: """
    if with_second
        return (x_dot(x, y, px, py), y_dot(x, y, px, py), px_dot(x, y, px, py), py_dot(x, y, px, py), x_ddot(x, y, px, py), y_ddot(x, y, px, py))
    else
        return (x_dot(x, y, px, py), y_dot(x, y, px, py), px_dot(x, y, px, py), py_dot(x, y, px, py))
    end
end

function velocities_half_delta(x, y, px, py, delta_t)
    x_new = x - x_dot(x, y, px, py) * delta_t / 2
    y_new = y - y_dot(x, y, px, py) * delta_t / 2
    px_new = px - px_dot(x, y, px, py) * delta_t / 2
    py_new = py - py_dot(x, y, px, py) * delta_t / 2
    return (x_dot(x_new, y_new, px_new, py_new), y_dot(x_new, y_new, px_new, py_new))
end

function W(x, y)
    """Функция нормализации значения"""
    return log((abs(diff_u_x(x, y)) + abs(diff_u_y(x, y))), 10)
end

function h_const(x, y, px=0, py=0)
    """Подсчёт постоянной Якоби"""
    return 0.5 * (x_dot(x, y, px, py) ^ 2 + y_dot(x, y, px, py) ^ 2) - u(x, y)
end


(2, 0, -0.67415967357487, -2.4202809902986715, -0.67415967357487, -4.4202809902986715)

<h1 align="center"> Эпизод I<br> <b>Визуализация точек Лагранжа, полости Роша и сферы Хилла</b></h1>

_________________
Для наглядного представления точек Лагранжа нами была выбрана функция `imshow` из пакета `matplotlib.pyplot`. Она задаёт изображение, как двумерный массив пикселей (в нашем коде массив `DAT`). Его заполнение осуществлялось с помощью перебора значений `i` от 0 до `(lim_x_max - lim_x_min) / step_x` и `j` от 0 до `(lim_y_max - lim_y_min) / step_y`. Каждому пикселю `DAT[i][j]` присваивалось значение `W(i, j)`. Для большей наглядности результата изображение было выведено в чёрно-белом формате `cmap='gray'`


<h1 align="center"> Эпизод II<br> <b>Нахождение координат точек Лагранжа</b></h1>

_______________
Для нахождения координат точек Лагранжа, нами использовались массивы,
```
mx = np.arange(lim_x_min, lim_x_max, step_x)
my = np.arange(lim_y_min, lim_y_max, step_y)
```
которые задаются минимальными значениями, максимальными значениями и шагом изменения с помощью функции `numpy.arrange`. В итоговый массивы точек Лагранжа `ans_x` и `ans_y` заносились координаты, в которых $ \frac{∂}{\partial x}U(x, y) = 0$ и $ \frac{∂}{\partial y}U(x, y) = 0$ (реализовано с помощью приращений и теоремы Лагранжа)
Из массива итоговых точек были исключены координаты центра масс и тел, и получившиеся координаты были добавлены в массивы `L_x` и `L_y` и отсортированы по номеру точки.  
Координаты этих точек также задаются формулами:

\begin{gather}
L_1 \:(R(1 - \sqrt[3]{\frac{\mu}{3}},\; 0))\\
L_2 \:(R(1 + \sqrt[3]{\frac{\mu}{3}}, \; 0)\\
L_3 \:(R(1 - \frac{5}{12}\mu, \; 0)\\
L_4 \:(\frac{R}{2} \cdot \frac{m_1-m_2}{m_1+m_2}, \; \frac{\sqrt[3]{3}}{2}R)\\
L_5 \:(\frac{R}{2} \cdot \frac{m_1-m_2}{m_1+m_2}, \; -\frac{\sqrt[3]{3}}{2}R)\\
\end{gather}

где:  

* $m_1$ - масса большего тела  
* $m_2$ - масса меньшего тела   
* $R$ - расстояние между телами


In [43]:
DAT = []
lim_x_min, lim_x_max = -2, 2
step_x = 0.01
lim_y_min, lim_y_max = -2, 2
step_y = 0.01
for (n, x) in enumerate(lim_x_min:step_x:lim_x_max)
    push!(DAT, [])
    for y in lim_y_min:step_y:lim_y_max
        push!(DAT[n], W(x, y))
    end
end
Gray(DAT)

LoadError: ignored